# Computation of the RER CF for land use

Based on country-specific CFs, we compute the one of RER which is an average weighted by the surface area of each country (i.e., similarly as the global average was computed in the original method).

In [1]:
import pandas as pd
import country_converter as coco

In [2]:
country_surface = pd.read_csv('data/world_bank_country_surface_area.csv')
ef_cfs = pd.read_excel('data/EF-LCIAMethod_CF(EF-v3.1).xlsx', sheet_name='lciamethods_CF', keep_default_na=False, na_values='')

In [3]:
country_surface['Country code (ISO2)'] = coco.convert(names=list(country_surface['Country Name']), to='ISO2')

Africa Eastern and Southern not found in regex
Africa Western and Central not found in regex
Arab World not found in regex
Central Europe and the Baltics not found in regex
Channel Islands not found in regex
Caribbean small states not found in regex
East Asia & Pacific ( not found in regex
Early-demographic dividend not found in regex
East Asia & Pacific not found in regex
Europe & Central Asia ( not found in regex
Europe & Central Asia not found in regex
Euro area not found in regex
European Union not found in regex
Fragile and conflict affected situations not found in regex
High income not found in regex
Heavily indebted poor countries (HIPC) not found in regex
IBRD only not found in regex
IDA & IBRD total not found in regex
IDA total not found in regex
IDA blend not found in regex
IDA only not found in regex
Not classified not found in regex
Latin America & Caribbean ( not found in regex
Latin America & Caribbean not found in regex
Least developed countries: UN classification not fo

In [4]:
ef_cfs_lu = ef_cfs[ef_cfs['LCIAMethod_name'] == 'Land use']

In [5]:
european_countries_full_names = [
    'Liechtenstein',
    'Armenia',
    'Lithuania',
    'Austria',
    'Luxembourg',
    'Azerbaijan',
    'Malta',
    'Belarus',
    'Moldova',
    'Belgium',
    'Monaco',
    'Bosnia and Herzegovina',
    'Montenegro',
    'Bulgaria',
    'Netherlands',
    'Croatia',
    'Norway',
    'Cyprus',
    'Poland',
    'Czech Republic',
    'Portugal',
    'Denmark',
    'Romania',
    'Estonia',
    'Russia',
    'Finland',
    'San Marino',
    'Former Yugoslav Republic of Macedonia',
    'Serbia',
    'France',
    'Slovakia',
    'Georgia',
    'Slovenia',
    'Germany',
    'Spain',
    'Greece',
    'Sweden',
    'Hungary',
    'Sweden',
    'Iceland',
    'Switzerland',
    'Ireland',
    'Turkey',
    'Italy',
    'Ukraine',
    'Kosovo',
    'United Kingdom',
]

In [6]:
european_countries_iso2 = coco.convert(names=european_countries_full_names, to='ISO2')

In [7]:
ef_cfs_lu_europe = ef_cfs_lu[ef_cfs_lu['LCIAMethod_location'].isin(european_countries_iso2)]

In [8]:
ef_cfs_lu_europe = ef_cfs_lu_europe.merge(country_surface[['Country code (ISO2)', '2022']], how='left', left_on='LCIAMethod_location', right_on='Country code (ISO2)')

In [9]:
# If a few locations have no surface area data, they are neglected
print(ef_cfs_lu_europe[ef_cfs_lu_europe['2022'].isna()].LCIAMethod_location.unique())
ef_cfs_lu_europe.dropna(subset=['2022'], inplace=True)

[]


In [10]:
def weighted_average(df, value_column, weight_column):
    return (df[value_column] * df[weight_column]).sum() / df[weight_column].sum()

In [11]:
# Group by "FLOW_name" and apply the weighted average function
weighted_avg_df = ef_cfs_lu_europe.groupby('FLOW_name').apply(
    lambda x: weighted_average(x, 'CF EF3.1', '2022')
).reset_index(name='CF EF3.1')

C:\Users\matth\AppData\Local\Temp\ipykernel_56364\1614582738.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_df = ef_cfs_lu_europe.groupby('FLOW_name').apply(


In [12]:
weighted_avg_df.to_csv('data/land_use_rer_cf.csv', index=False)

In [13]:
weighted_avg_df

,FLOW_name,CF EF3.1
0,agriculture,28.777402
1,"agriculture, mosaic",28.777402
2,arable,29.036045
3,"arable, fallow",29.294748
4,"arable, flooded crops",28.003243
...,...,...
166,"urban, green areas",36.115544
167,urban/industrial fallow,131.437295
168,wetlands,-22.280758
169,"wetlands, coastal",-22.280758


In [14]:
# Verification
ef_cfs_lu_europe_agri = ef_cfs_lu_europe[ef_cfs_lu_europe['FLOW_name'] == 'agriculture']
ef_cfs_lu_europe_agri['Weighted CF'] = ef_cfs_lu_europe_agri['CF EF3.1'] * ef_cfs_lu_europe_agri['2022']
rer_cf = ef_cfs_lu_europe_agri['Weighted CF'].sum() / ef_cfs_lu_europe_agri['2022'].sum()
rer_cf

C:\Users\matth\AppData\Local\Temp\ipykernel_56364\2711157756.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ef_cfs_lu_europe_agri['Weighted CF'] = ef_cfs_lu_europe_agri['CF EF3.1'] * ef_cfs_lu_europe_agri['2022']


28.777402327377903